模型设计的代码需要用到上一节数据处理的Python类，定义如下：

In [21]:
import pandas as pd
import random
import numpy as np
import paddle
from paddle.nn import Linear, Embedding, Conv2D
import paddle.nn.functional as F
import math
from PIL import Image

In [148]:
class MovieLen(object):
    def __init__(self, use_poster):
        self.use_poster = use_poster
        # 声明每个数据文件的路径
        usr_info_path = "/Users/wlin109/Desktop/awesome-DeepLearning/junior_class/chapter-7-Recommendation_System/notebook/data_freelancer/freelancers.xlsx"
        if use_poster:
            rating_path = "./work/ml-1m/new_rating.txt"
        else:
            rating_path = "/Users/wlin109/Desktop/awesome-DeepLearning/junior_class/chapter-7-Recommendation_System/notebook/data_freelancer/post_rating.xlsx"

        job_info_path = "/Users/wlin109/Desktop/awesome-DeepLearning/junior_class/chapter-7-Recommendation_System/notebook/data_freelancer/jobs.xlsx"
        self.poster_path = "./work/ml-1m/posters/"
        # 得到电影数据
        self.job_info, self.job_city = self.get_job_info(job_info_path)
        # 记录电影的最大ID
        # self.max_job = np.max([self.movie_cat[k] for k in self.movie_cat])
        # self.max_mov_tit = np.max([self.movie_title[k] for k in self.movie_title])
        self.max_job_id = np.max(list(map(int, self.job_info.keys())))
        self.max_job_city = np.max(list(map(int, self.job_city.values())))
        # 记录用户数据的最大ID
        self.max_usr_id = 0
        # 得到用户数据
        self.usr_info, self.usr_title_info, self.usr_state_info = self.get_usr_info(usr_info_path)
        self.max_usr_title = np.max([self.usr_title_info[k] for k in self.usr_title_info])
        self.max_usr_state = np.max(list(map(int, self.usr_state_info.values())))
        # print(self.usr_info)
        # 得到评分数据
        self.rating_info = self.get_rating_info(rating_path)
        # 构建数据集 
        self.dataset = self.get_dataset(usr_info=self.usr_info,
                                        job_info=self.job_info,
                                        rating_info=self.rating_info
                                       )
        # 划分数据集，获得数据加载器
        self.train_dataset = self.dataset[:int(len(self.dataset)*0.9)]
        self.valid_dataset = self.dataset[int(len(self.dataset)*0.9):]
        print("##Total dataset instances: ", len(self.dataset))
        print("##MovieLens dataset information: \nusr num: {}\n"
              "job num: {}".format(len(self.usr_info),len(self.job_info)))
    # 得到电影数据
    def get_job_info(self, path):
        # 打开文件，编码方式选择ISO-8859-1，读取所有数据到data中 
        df_job = pd.read_excel(open(path, 'rb'),
                             sheet_name='summary')
        # 建立三个字典，分别用户存放电影所有信息，电影的名字信息、类别信息
        job_info, job_city = {}, {}
        df_job['Value'].replace(['Min of 1 year','Min of 3 years','Min of 5 years','Min of 7 years','Min of 9 years'],[1,3,5,7,9],inplace= True)
        # 对电影名字、类别中不同的单词计数
        count_c = 1
        
        for i in df_job.index:
            job_id = df_job['jobid'][i]
            
            if df_job['WorkLocationCity'][i] not in job_city:
                job_city[df_job['WorkLocationCity'][i]] = count_c
                count_c +=1
            job_info[job_id] = {'job_id': int(job_id),
                                'job_city': job_city[df_job['WorkLocationCity'][i]],
                                'job_exp_year': df_job['Value'][i]
                                }


        return job_info, job_city

    def get_usr_info(self, path):
        # 打开文件，读取所有行到data中
        data = pd.read_excel(open(path, 'rb'),
                             sheet_name='freelancer summary') 
        # 建立用户信息的字典
        state_info, title_info, usr_info = {}, {}, {}
        max_state_len, max_title_len= 0,0
        count_s, count_t = 1,1
        data['State'] = data['State'].fillna('')
        data['Title'] = data['Title'].mask(pd.to_numeric(data['Title'], errors='coerce').notna()).fillna('')
        data['Title'].replace(['/','& ','|','- ',', ','Sr.'],[' ',' ',' ',' ',' ','Senior'], inplace = True)
        
        
        for i in data.index:
            usr_id = data['FreelancerID'][i]
            
            each_title = data['Title'][i].split()
            # print(each_title)
            max_title_len = max(max_title_len, len(each_title))

            if data['State'][i] not in state_info:
                state_info[data['State'][i]] = count_s
                count_s += 1

            for t in each_title:
                if t not in title_info:
                    title_info[t] = count_t
                    count_t +=1

            v_title = [title_info[k] for k in each_title]
            # print(usr_id,v_title)
            while len(v_title)<39:
                v_title.append(0)
            # print(usr_id)
            usr_info[usr_id] = {'usr_id': int(usr_id),
                                 'title': v_title,
                                 'state': state_info[data['State'][i]]

                                }
            self.max_usr_id = max(self.max_usr_id, int(usr_id))

        return usr_info, title_info, state_info
    # 得到评分数据
    def get_rating_info(self, path):
        # 打开文件，读取所有行到data中
        df_rating = pd.read_excel(open(path, 'rb'))
        # 创建一个字典
        rating_info = {}
        for i in df_rating.index:
            usr_id, jobid, rating = df_rating['FreelancerID'][i], df_rating['jobid'][i],df_rating['rating'][i]
            if usr_id in self.usr_info.keys():
                if usr_id not in rating_info.keys():
                    rating_info[usr_id] = {jobid:float(rating)}
                else:
                    rating_info[usr_id][jobid] = float(rating)
        return rating_info
    # 构建数据集
    def get_dataset(self, usr_info, job_info, rating_info):
        trainset = []
        # 按照评分数据的key值索引数据
        for usr_id in rating_info.keys():
            usr_ratings = rating_info[usr_id]
            for jobid in usr_ratings:
                trainset.append({'usr_info': usr_info[usr_id],
                                 'job_info': job_info[jobid],
                                 'scores': usr_ratings[jobid]})
        return trainset
    
    def load_data(self, dataset=None, mode='train'):
        use_poster = False

        # 定义数据迭代Batch大小
        BATCHSIZE = 256

        data_length = len(dataset)
        index_list = list(range(data_length))
        # 定义数据迭代加载器
        def data_generator():
            # 训练模式下，打乱训练数据
            if mode == 'train':
                random.shuffle(index_list)
            # 声明每个特征的列表
            usr_id_list,usr_title_list,usr_state_list,usr_job_list = [], [], [], []
            job_id_list,job_city_list,job_exp_year_list = [], [], []
            score_list = []
            # 索引遍历输入数据集
            for idx, i in enumerate(index_list):
                # 获得特征数据保存到对应特征列表中
                usr_id_list.append(dataset[i]['usr_info']['usr_id'])
                usr_title_list.append(dataset[i]['usr_info']['title'])
                usr_state_list.append(dataset[i]['usr_info']['state'])
                # usr_job_list.append(dataset[i]['usr_info']['job'])

                job_id_list.append(dataset[i]['job_info']['job_id'])
                job_city_list.append(dataset[i]['job_info']['job_city'])
                job_exp_year_list.append(dataset[i]['job_info']['job_exp_year'])
                job_id = dataset[i]['job_info']['job_id']

                if use_poster:
                    # 不使用图像特征时，不读取图像数据，加快数据读取速度
                    poster = Image.open(poster_path+'mov_id{}.jpg'.format(str(mov_id)))
                    poster = poster.resize([64, 64])
                    if len(poster.size) <= 2:
                        poster = poster.convert("RGB")

                    mov_poster_list.append(np.array(poster))

                score_list.append(int(dataset[i]['scores']))
                # 如果读取的数据量达到当前的batch大小，就返回当前批次
                
                if len(usr_id_list)==BATCHSIZE:
                    # 转换列表数据为数组形式，reshape到固定形状
                    usr_id_arr = np.array(usr_id_list)
                    usr_title_arr = np.reshape(np.array(usr_title_list), [BATCHSIZE, 1, 39]).astype(np.int64)
                    usr_state_arr = np.array(usr_state_list)
                    # usr_job_arr = np.array(usr_job_list)

                    job_id_arr = np.array(job_id_list)
                    job_city_arr = np.array(job_city_list)
                    job_exp_year_arr = np.array(job_exp_year_list)

                    if use_poster:
                        mov_poster_arr = np.reshape(np.array(mov_poster_list)/127.5 - 1, [BATCHSIZE, 3, 64, 64]).astype(np.float32)
                    else:
                        mov_poster_arr = np.array([0.])

                    scores_arr = np.reshape(np.array(score_list), [-1, 1]).astype(np.float32)

                    # 返回当前批次数据
                    yield [usr_id_arr, usr_title_arr, usr_state_arr], \
                           [job_id_arr, job_city_arr, job_exp_year_arr], scores_arr

                    # 清空数据
                    usr_id_list, usr_title_list, usr_state_list = [], [], []
                    job_id_list,job_city_list,job_exp_year_list = [], [], []
                    score_list = []
                    mov_poster_list = []
        return data_generator

    # 声明数据读取类

    


In [ ]:
dataset = MovieLen(False)


# 定义数据读取器
train_loader = dataset.load_data(dataset=dataset.train_dataset, mode='train')
# 迭代的读取数据， Batchsize = 256
for idx, data in enumerate(train_loader()):
    usr, job, score = data
    print("打印用户ID，title,state，数据的维度：")
    for v in usr:
        print(v.shape)
    print("打印jobID，名字，类别数据的维度：")
    for v in job:
        print(v.shape)
    break


In [145]:
class Model(paddle.nn.Layer):
    def __init__(self,use_poster, use_usr_title, use_usr_state, use_job_city, use_job_exp_year,fc_sizes):
        super(Model, self).__init__()
        
        # 将传入的name信息和bool型参数添加到模型类中
        self.use_mov_poster = use_poster
        self.use_usr_title = use_usr_title
        self.use_usr_state = use_usr_state
        self.use_job_city = use_job_city
        self.use_job_exp_year = use_job_exp_year
        self.fc_sizes = fc_sizes
        
        # 获取数据集的信息，并构建训练和验证集的数据迭代器
        Dataset = MovieLen(self.use_mov_poster)
        self.Dataset = Dataset
        self.trainset = self.Dataset.train_dataset
        self.valset = self.Dataset.valid_dataset
        self.train_loader = self.Dataset.load_data(dataset=self.trainset, mode='train')
        self.valid_loader = self.Dataset.load_data(dataset=self.valset, mode='valid')

        """ define network layer for embedding usr info """
        USR_ID_NUM = Dataset.max_usr_id + 1
        # 对用户ID做映射，并紧接着一个Linear层
        self.usr_emb = Embedding(num_embeddings=USR_ID_NUM, embedding_dim=32, sparse=False)
        self.usr_fc = Linear(in_features=32, out_features=32)
        
        # 对usr title 信息做映射，并紧接着一个Linear层
        USR_TITLE_DICT_SIZE = len(Dataset.usr_title_info) + 1
        self.usr_title_emb = Embedding(num_embeddings=USR_TITLE_DICT_SIZE, embedding_dim=32, sparse=False)
        self.usr_title_conv = Conv2D(in_channels=1, out_channels=1, kernel_size=(3, 1), stride=(2,1), padding=0)
        self.usr_title_conv2 = Conv2D(in_channels=1, out_channels=1, kernel_size=(3, 1), stride=1, padding=0)
        
        # 对usr state信息做映射，并紧接着一个Linear层
        USR_STATE_DICT_SIZE = Dataset.max_usr_state + 1
        self.usr_state_emb = Embedding(num_embeddings=USR_STATE_DICT_SIZE, embedding_dim=16)
        self.usr_state_fc = Linear(in_features=16, out_features=16)
        
        # # 对用户职业信息做映射，并紧接着一个Linear层
        # USR_JOB_DICT_SIZE = Dataset.max_usr_job + 1
        # self.usr_job_emb = Embedding(num_embeddings=USR_JOB_DICT_SIZE, embedding_dim=16)
        # self.usr_job_fc = Linear(in_features=16, out_features=16)
        
        # 新建一个Linear层，用于整合用户数据信息
        self.usr_combined = Linear(in_features=80, out_features=200)
        
        """ define network layer for embedding job info """
        # 对电影ID信息做映射，并紧接着一个Linear层
        JOB_DICT_SIZE = Dataset.max_job_id + 1
        self.job_emb = Embedding(num_embeddings=JOB_DICT_SIZE, embedding_dim=32)
        self.job_fc = Linear(in_features=32, out_features=32)
        
        # 对电影类别做映射
        JOB_CITY_DICT_SIZE = Dataset.max_job_city + 1
        self.job_city_emb = Embedding(num_embeddings=JOB_CITY_DICT_SIZE, embedding_dim=16, sparse=False)
        self.job_city_fc = Linear(in_features=16, out_features=16)
        
        # 对电影名称做映射
        JOB_EXP_YEAR_DICT_SIZE = 9 + 1
        self.job_exp_year_emb = Embedding(num_embeddings=JOB_EXP_YEAR_DICT_SIZE, embedding_dim=16, sparse=False)
        self.job_exp_year_fc = Linear(in_features=16, out_features=16)
        
        # 新建一个FC层，用于整合电影特征
        self.job_concat_embed = Linear(in_features=64, out_features=200)

        user_sizes = [200] + self.fc_sizes
        acts = ["relu" for _ in range(len(self.fc_sizes))]
        self._user_layers = []
        for i in range(len(self.fc_sizes)):
            linear = paddle.nn.Linear(
                in_features=user_sizes[i],
                out_features=user_sizes[i + 1],
                weight_attr=paddle.ParamAttr(
                    initializer=paddle.nn.initializer.Normal(
                        std=1.0 / math.sqrt(user_sizes[i]))))
            self._user_layers.append(linear)
            if acts[i] == 'relu':
                act = paddle.nn.ReLU()
                self._user_layers.append(act)
        
        #电影特征和用户特征使用了不同的全连接层，不共享参数
        job_sizes = [200] + self.fc_sizes
        acts = ["relu" for _ in range(len(self.fc_sizes))]
        self._job_layers = []
        for i in range(len(self.fc_sizes)):
            linear = paddle.nn.Linear(
                in_features=job_sizes[i],
                out_features=job_sizes[i + 1],
                weight_attr=paddle.ParamAttr(
                    initializer=paddle.nn.initializer.Normal(
                        std=1.0 / math.sqrt(job_sizes[i]))))
            self._job_layers.append(linear)
            if acts[i] == 'relu':
                act = paddle.nn.ReLU()
                self._job_layers.append(act)
        
    # 定义计算用户特征的前向运算过程
    def get_usr_feat(self, usr_var):
        """ get usr features"""
        # 获取到用户数据
        usr_id, usr_title, usr_state = usr_var
        # 将用户的ID数据经过embedding和Linear计算，得到的特征保存在feats_collect中
        feats_collect = []
        batch_size = usr_id.shape[0]
        usr_id = self.usr_emb(usr_id)
        usr_id = self.usr_fc(usr_id)
        usr_id = F.relu(usr_id)
        feats_collect.append(usr_id)

        
        if self.use_usr_title:
            # 计算电影名字的特征映射，对特征映射使用卷积计算最终的特征
            usr_title = self.usr_title_emb(usr_title)
            usr_title = F.relu(self.usr_title_conv2(F.relu(self.usr_title_conv(usr_title))))
            usr_title = paddle.sum(usr_title, axis=2, keepdim=False)
            usr_title = F.relu(usr_title)
            usr_title = paddle.reshape(usr_title, [batch_size, -1])
            feats_collect.append(usr_title)
        
        
        # 选择是否使用用户的年龄-职业特征
        if self.use_usr_state:
            # 计算用户的年龄特征，并保存在feats_collect中
            usr_state = self.usr_state_emb(usr_state)
            usr_state = self.usr_state_fc(usr_state)
            usr_state = F.relu(usr_state)
            feats_collect.append(usr_state)
        
        # 将用户的特征级联，并通过Linear层得到最终的用户特征
        usr_feat = paddle.concat(feats_collect, axis=1)
        user_features = F.tanh(self.usr_combined(usr_feat))
        #通过3层全连接层，获得用于计算相似度的用户特征
        for n_layer in self._user_layers:
            user_features = n_layer(user_features)

        return user_features

        # 定义电影特征的前向计算过程
    def get_job_feat(self, job_var):
        """ get movie features"""
        # 获得电影数据
        job_id, job_city, job_exp_year= job_var
        feats_collect = []
        # 获得batchsize的大小
        batch_size = job_id.shape[0]
        # 计算电影ID的特征，并存在feats_collect中
        job_id = self.job_emb(job_id)
        job_id = self.job_fc(job_id)
        job_id = F.relu(job_id)
        feats_collect.append(job_id)
        
        # 如果使用电影的种类数据，计算电影种类特征的映射
        if self.use_job_city:
            # 计算电影种类的特征映射，对多个种类的特征求和得到最终特征
            job_city = self.job_city_emb(job_city)
            job_city = self.job_city_fc(job_city)
            job_city = F.relu(job_city)
            feats_collect.append(job_city)

        if self.use_job_exp_year:
            # 计算电影名字的特征映射，对特征映射使用卷积计算最终的特征
            job_exp_year = self.job_exp_year_emb(job_exp_year)
            job_exp_year = self.job_exp_year_fc(job_exp_year)
            job_exp_year = F.relu(job_exp_year)
            feats_collect.append(job_exp_year)
            
        # 使用一个全连接层，整合所有电影特征，映射为一个200维的特征向量
        job_feat = paddle.concat(feats_collect, axis=1)
        job_features = F.tanh(self.job_concat_embed(job_feat))
        #通过3层全连接层，获得用于计算相似度的电影特征
        for n_layer in self._job_layers:
            job_features = n_layer(job_features)

        return job_features
    
    # 定义个性化推荐算法的前向计算
    def forward(self, usr_var, job_var):
        # 计算用户特征和电影特征
        user_features = self.get_usr_feat(usr_var)
        job_features = self.get_job_feat(job_var)
        # print(user_features.shape, job_features.shape)

        # 根据计算的特征计算相似度
        res = F.common.cosine_similarity(user_features, job_features).reshape([-1,1])
        # 将相似度扩大范围到和电影评分相同数据范围
        # res = paddle.scale(res, scale=5)
        return user_features, job_features, res
   

In [ ]:
## 测试电影特征提取网络
fc_sizes=[128, 64, 32]
model = Model(use_poster=False,use_usr_title=True, use_usr_state=True, use_job_city=True, use_job_exp_year=True,fc_sizes=fc_sizes)
model.eval()

data_loader = model.train_loader

for idx, data in enumerate(data_loader()):
    # 获得数据，并转为动态图格式
    usr, job, score = data
    # 只使用每个Batch的第一条数据
    job_v = [var[0:1] for var in job]
    # print(job_v)
    _job_v = [np.squeeze(var[0:1]) for var in job]
    print("输入的JobID数据：{}\n城市数据：{} \n年限需求数据：{} ".format(*_job_v))
    job_v = [paddle.to_tensor(var) for var in job_v]
    job_feat = model.get_job_feat(job_v)
    print("计算得到的电影特征维度是：", job_feat.shape)
    
    # 只使用每个Batch的第一条数据
#     usr_v = [var[0:1] for var in usr]
    
#     _usr_v = [np.squeeze(var[0:1]) for var in usr]
#     print("输入的usrID数据：{}\nTITLE数据：{} \nSTATE需求数据：{} ".format(*_usr_v))
#     usr_v = [paddle.to_tensor(usr) for var in usr_v]
#     usr_feat = model.get_usr_feat(usr_v)
#     print("计算得到的usr特征维度是：", usr_feat.shape)
    
    break

In [120]:
for idx, data in enumerate(data_loader()):
    # 获得数据，并转为动态图格式
    usr, job, score = data
    usr_v = [var[0:1] for var in usr]
    print(usr_v)

    _usr_v = [np.squeeze(var[0:1]) for var in usr]
    print("输入的JobID数据：{}\n城市数据：{} \n年限需求数据：{} ".format(*_usr_v))
    usr_v = [paddle.to_tensor(var) for var in usr_v]
    usr_feat = model.get_usr_feat(usr_v)
    print("计算得到的电影特征维度是：", usr_feat.shape)
    break

In [122]:
F.common.cosine_similarity(usr_feat, job_feat)

In [150]:
def train(model):
    # 配置训练参数
    lr = 0.001
    Epoches = 20
    paddle.set_device('cpu') 

    # 启动训练
    model.train()
    # 获得数据读取器
    data_loader = model.train_loader
    # 使用adam优化器，学习率使用0.01
    opt = paddle.optimizer.Adam(learning_rate=lr, parameters=model.parameters())
    
    for epoch in range(0, Epoches):
        for idx, data in enumerate(data_loader()):
            # 获得数据，并转为tensor格式
            usr, mov, score = data
            usr_v = [paddle.to_tensor(var) for var in usr]
            mov_v = [paddle.to_tensor(var) for var in mov]
            scores_label = paddle.to_tensor(score)
            # 计算出算法的前向计算结果
            _, _, scores_predict = model(usr_v, mov_v)
            # 计算loss
            loss = F.square_error_cost(scores_predict, scores_label)
            avg_loss = paddle.mean(loss)

            if idx % 10 == 0:
                print("epoch: {}, batch_id: {}, loss is: {}".format(epoch, idx, avg_loss.numpy()))
                
            # 损失函数下降，并清除梯度
            avg_loss.backward()
            opt.step()
            opt.clear_grad()

        # 每个epoch 保存一次模型
        paddle.save(model.state_dict(), './checkpoint/epoch'+str(epoch)+'.pdparams')

In [151]:
fc_sizes=[128, 64, 32]
model = Model(use_poster=False,use_usr_title=True, use_usr_state=True, use_job_city=True, use_job_exp_year=True,fc_sizes=fc_sizes)
train(model)

##Total dataset instances:  20024
##MovieLens dataset information: 
usr num: 63025
job num: 8199
epoch: 0, batch_id: 0, loss is: [0.27118278]
epoch: 0, batch_id: 10, loss is: [0.24645521]
epoch: 0, batch_id: 20, loss is: [0.21586812]
epoch: 0, batch_id: 30, loss is: [0.20636056]
epoch: 0, batch_id: 40, loss is: [0.22544795]
epoch: 0, batch_id: 50, loss is: [0.20607668]
epoch: 0, batch_id: 60, loss is: [0.19542462]
epoch: 1, batch_id: 0, loss is: [0.15241402]
epoch: 1, batch_id: 10, loss is: [0.14415714]
epoch: 1, batch_id: 20, loss is: [0.16082124]
epoch: 1, batch_id: 30, loss is: [0.16417757]
epoch: 1, batch_id: 40, loss is: [0.15804133]
epoch: 1, batch_id: 50, loss is: [0.12346265]
epoch: 1, batch_id: 60, loss is: [0.18682002]
epoch: 2, batch_id: 0, loss is: [0.1163563]
epoch: 2, batch_id: 10, loss is: [0.10856698]
epoch: 2, batch_id: 20, loss is: [0.09622841]
epoch: 2, batch_id: 30, loss is: [0.11619576]
epoch: 2, batch_id: 40, loss is: [0.11683284]
epoch: 2, batch_id: 50, loss is: 

In [152]:
from math import sqrt
def evaluation(model, params_file_path):
    model_state_dict = paddle.load(params_file_path)
    model.load_dict(model_state_dict)
    model.eval()

    acc_set = []
    avg_loss_set = []
    squaredError=[]
    for idx, data in enumerate(model.valid_loader()):
        usr, mov, score_label = data
        usr_v = [paddle.to_tensor(var) for var in usr]
        mov_v = [paddle.to_tensor(var) for var in mov]

        _, _, scores_predict = model(usr_v, mov_v)
        pred_scores = scores_predict.numpy()
        
        avg_loss_set.append(np.mean(np.abs(pred_scores - score_label)))
        squaredError.extend(np.abs(pred_scores - score_label)**2)

        diff = np.abs(pred_scores - score_label)
        diff[diff>0.5] = 1
        acc = 1 - np.mean(diff)
        acc_set.append(acc)
    RMSE=sqrt(np.sum(squaredError) / len(squaredError))
    
    # print("RMSE = ", sqrt(np.sum(squaredError) / len(squaredError)))#均方根误差RMSE
        
    return np.mean(acc_set), np.mean(avg_loss_set),RMSE

In [155]:
param_path = "./checkpoint/epoch"
for i in range(20):
    acc, mae,RMSE = evaluation(model, param_path+str(i)+'.pdparams')
    print("no:",i,"ACC:", acc, "MAE:", mae,'RMSE:',RMSE)

no: 0 ACC: 0.49630554233278545 MAE: 0.3990512 RMSE: 0.4518680144295593
no: 1 ACC: 0.5179521186011178 MAE: 0.38599536 RMSE: 0.46779810211569234
no: 2 ACC: 0.536872216633388 MAE: 0.37615007 RMSE: 0.4761036935701747
no: 3 ACC: 0.5462229081562587 MAE: 0.37712243 RMSE: 0.4950257897092257
no: 4 ACC: 0.5516033172607422 MAE: 0.37069663 RMSE: 0.50065746677657
no: 5 ACC: 0.5630768324647631 MAE: 0.3681319 RMSE: 0.5022743134732185
no: 6 ACC: 0.5829110656465802 MAE: 0.3555691 RMSE: 0.4961206280836596
no: 7 ACC: 0.5693751914160592 MAE: 0.3640159 RMSE: 0.4996352397930526
no: 8 ACC: 0.5662841584001269 MAE: 0.36337328 RMSE: 0.49569641610324827
no: 9 ACC: 0.5749909579753876 MAE: 0.3575831 RMSE: 0.49407045198905497
no: 10 ACC: 0.5615896199430738 MAE: 0.36656183 RMSE: 0.4924385658769846
no: 11 ACC: 0.5761820673942566 MAE: 0.35617608 RMSE: 0.49065357364549816
no: 12 ACC: 0.5813469077859607 MAE: 0.35421118 RMSE: 0.48893387259501736
no: 13 ACC: 0.5737074783870152 MAE: 0.35866183 RMSE: 0.48968861075066755
no: